In [43]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

import sys
sys.path.append("../")
import os
import concurrent.futures
from queue import Queue # Python 3.x

import re
import datetime
import pandas as pd
import pandas_market_calendars as mcal
from polygon import RESTClient
#from tqdm.notebook import tqdm
import logging
import threading
import time
import pytz
eastern = 'US/Eastern'

In [44]:
que = Queue()  

In [4]:
key = "qRFktbNL6A_WzaFhqR80IllApsmjPY6a"

def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')

def get_split_data_for_stock(stock):
    with RESTClient(key) as client:
        try:
            respex = client.reference_stock_splits(str(stock).upper())
            result = respex.__dict__['results']
            return result
        except:
            print("Error getting split data for "+ str(stock))
            return []

def get_dividend_data_for_stock(stock):
    with RESTClient(key) as client:
        try:
            respex = client.reference_stock_dividends(str(stock).upper())
            result = respex.__dict__['results']
            return result
        except:
            print("Error getting dividend data for "+ str(stock))
            return []
        
def extract_qualified_symbols(df, plvl, advlvl):
    qualified = df[(df.p_lvl == plvl) & (df.adv_lvl == advlvl)]
    return qualified

In [46]:
def threaded(func):
    """
    Decorator that multithreads the target function
    with the given parameters. Returns the thread
    created for the function
    """
    def wrapper(*args, **kwargs):
        thread = Thread(target=func, args=args)
        thread.start()
        return thread
    return wrapper

In [47]:
@threaded
def download_data(q_sym_list, result):
    splits_df = []
    dividend_df  = []
    for i in tqdm(range(len(q_sym_list))):
    
        ticker = q_sym_list[i]

        sp = get_split_data_for_stock(ticker)
        div = get_dividend_data_for_stock(ticker)
        a = pd.DataFrame(sp)
        b = pd.DataFrame(div)

        splits_df.append(a)

        dividend_df.append(b)

        time.sleep(0.1)
    result.append([pd.concat(splits_df), pd.concat(dividend_df)])
    return [pd.concat(splits_df), pd.concat(dividend_df)]

In [48]:
main_df = pd.read_csv('stocks-profile.csv')


In [49]:
q_sym_list = []
for i in range(5):
    q_sym = extract_qualified_symbols(main_df, i, 2)
    q_sym_list.append(list(q_sym['symbol']))

In [50]:
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

In [51]:
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

threads = []
results = []
for i in range(len(q_sym_list)):
    threads.append(download_data(q_sym_list[i], results))


# wait for all threads to finish before moving on
for thread in threads:
    thread.join()

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")


Exception in thread Thread-28:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-46-13a1fbb2a299>", line 8, in <lambda>
    thread = threading.Thread(target=lambda func, arg1: func.put(foo(arg1)), args=args)
AttributeError: 'list' object has no attribute 'put'

Exception in thread Thread-29:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-46-13a1fbb2a299>", line 8, in <lambda>
    thread = threading.Thread(target=lambda func, arg1: func.put(foo(arg1)), args=args)
AttributeError: 'list' object has no attribute 'put'

Exception in thread Thread-30:
Trace

In [41]:
pd.concat(results[2])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,amount,declaredDate,exDate,forfactor,paymentDate,ratio,recordDate,ticker,tofactor
0,NaN,2016-11-09,2016-11-09,1.0,2016-11-09,3.000000,NaN,GNUS,3.0
0,NaN,NaN,2000-04-11,NaN,2000-04-11,0.250000,NaN,NOK,NaN
1,NaN,NaN,1999-04-12,NaN,1999-04-12,0.500000,NaN,NOK,NaN
2,NaN,NaN,1998-04-16,NaN,1998-04-16,0.500000,NaN,NOK,NaN
0,NaN,2020-08-10,2020-08-10,1.0,2020-08-10,25.000000,NaN,TOPS,25.0
1,NaN,2019-08-22,2019-08-22,1.0,2019-08-22,20.000000,NaN,TOPS,20.0
2,NaN,2018-03-26,2018-03-26,1.0,2018-03-26,10.000000,NaN,TOPS,10.0
3,NaN,2017-10-06,2017-10-06,1.0,2017-10-06,2.000000,NaN,TOPS,2.0
4,NaN,2017-08-03,2017-08-03,1.0,2017-08-03,30.000000,NaN,TOPS,30.0
5,NaN,2017-06-23,2017-06-23,1.0,2017-06-23,15.000000,NaN,TOPS,15.0
